In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from keras.models import model_from_json

Using TensorFlow backend.


## Read and prepare data

In [2]:
num_classes = 10
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Define Architecture

In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

## Archtiecture

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

## Training

In [5]:
batch_size = 64 
epochs = 20 

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
315s - loss: 0.2740 - acc: 0.9175 - val_loss: 0.0736 - val_acc: 0.9774
Epoch 2/20
351s - loss: 0.1023 - acc: 0.9699 - val_loss: 0.0509 - val_acc: 0.9839
Epoch 3/20
358s - loss: 0.0790 - acc: 0.9773 - val_loss: 0.0441 - val_acc: 0.9854
Epoch 4/20
357s - loss: 0.0672 - acc: 0.9804 - val_loss: 0.0381 - val_acc: 0.9867
Epoch 5/20
347s - loss: 0.0585 - acc: 0.9826 - val_loss: 0.0340 - val_acc: 0.9885
Epoch 6/20
368s - loss: 0.0539 - acc: 0.9843 - val_loss: 0.0345 - val_acc: 0.9886
Epoch 7/20
378s - loss: 0.0497 - acc: 0.9853 - val_loss: 0.0340 - val_acc: 0.9888
Epoch 8/20
339s - loss: 0.0458 - acc: 0.9863 - val_loss: 0.0337 - val_acc: 0.9891
Epoch 9/20
313s - loss: 0.0441 - acc: 0.9874 - val_loss: 0.0313 - val_acc: 0.9900
Epoch 10/20
302s - loss: 0.0433 - acc: 0.9873 - val_loss: 0.0351 - val_acc: 0.9893
Epoch 11/20
302s - loss: 0.0416 - acc: 0.9872 - val_loss: 0.0321 - val_acc: 0.9895
Epoch 12/20
303s - loss: 0.0397 - acc: 0.9887 

## Prediction

In [6]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("CNN Error: %.2f%%" % (100-score[1]*100))

Test loss: 0.0314161958387
Test accuracy: 0.9903
CNN Error: 0.97%


In [7]:
model_json = model.to_json()
with open("models/mnist_97.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("models/mnist_97.h5")
print("Saved model to disk")


# load json and create model
json_file = open('models/mnist_97.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("models/mnist_97.h5")
print("Loaded model from disk")

Saved model to disk
Loaded model from disk
